# OpenAI APIs - Completions

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/api-reference).

This tutorial covers the following popular APIs:

- `chat/completions`
- `completions`
- `batches`

Check out other tutorials to learn about vision APIs for vision-language models and embedding APIs for embedding models.

## Launch A Server

This code block is equivalent to executing 

```bash
python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
--port 30000 --host 0.0.0.0
```

in your terminal and wait for the server to be ready.

In [1]:
from sglang.utils import (
    execute_shell_command,
    wait_for_server,
    terminate_process,
    print_highlight,
)

server_process = execute_shell_command(
    "python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct --port 30000 --host 0.0.0.0"
)

wait_for_server("http://localhost:30000")

[2024-11-03 07:22:23] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, is_embedding=False, host='0.0.0.0', port=30000, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, tp_size=1, stream_interval=1, random_seed=21016084, constrained_json_whitespace_pattern=None, decode_log_interval=40, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, api_key=None, file_storage_pth='SGLang_storage', enable_cache_report=False, watchdog_timeout=600, dp_size=1, load_balance_method='round_robin', dist_in

[2024-11-03 07:22:39 TP0] Init torch distributed begin.


[2024-11-03 07:22:39 TP0] Load weight begin. avail mem=78.59 GB


[2024-11-03 07:22:40 TP0] lm_eval is not installed, GPTQ may not be usable


INFO 11-03 07:22:40 weight_utils.py:243] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.60it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.47it/s]

[2024-11-03 07:22:43 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.bfloat16, avail mem=63.50 GB
[2024-11-03 07:22:43 TP0] Memory pool end. avail mem=8.37 GB
[2024-11-03 07:22:43 TP0] Capture cuda graph begin. This can take up to several minutes.


[2024-11-03 07:22:51 TP0] max_total_num_tokens=442913, max_prefill_tokens=16384, max_running_requests=2049, context_len=131072


[2024-11-03 07:22:52] INFO:     Started server process [2204938]
[2024-11-03 07:22:52] INFO:     Waiting for application startup.
[2024-11-03 07:22:52] INFO:     Application startup complete.
[2024-11-03 07:22:52] INFO:     Uvicorn running on http://0.0.0.0:30000 (Press CTRL+C to quit)


[2024-11-03 07:22:52] INFO:     127.0.0.1:42618 - "GET /v1/models HTTP/1.1" 200 OK
[2024-11-03 07:22:53] INFO:     127.0.0.1:42628 - "GET /get_model_info HTTP/1.1" 200 OK
[2024-11-03 07:22:53 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-11-03 07:22:53] INFO:     127.0.0.1:42636 - "POST /generate HTTP/1.1" 200 OK
[2024-11-03 07:22:53] The server is fired up and ready to roll!


## Chat Completions

### Usage

The server fully implements the OpenAI API.
It will automatically apply the chat template specified in the Hugging Face tokenizer, if one is available.
You can also specify a custom chat template with `--chat-template` when launching the server.

In [2]:
import openai

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant"},
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2024-11-03 07:22:58 TP0] Prefill batch. #new-seq: 1, #new-token: 48, #cached-token: 1, cache hit rate: 1.79%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-11-03 07:22:58 TP0] Decode batch. #running-req: 1, #token: 82, token usage: 0.00, gen throughput (token/s): 5.69, #queue-req: 0
[2024-11-03 07:22:58] INFO:     127.0.0.1:42640 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Parameters

The chat completions API accepts OpenAI Chat Completions API's parameters. Refer to [OpenAI Chat Completions API](https://platform.openai.com/docs/api-reference/chat/create) for more details.

Here is an example of a detailed chat completion request:

In [3]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a knowledgeable historian who provides concise responses.",
        },
        {"role": "user", "content": "Tell me about ancient Rome"},
        {
            "role": "assistant",
            "content": "Ancient Rome was a civilization centered in Italy.",
        },
        {"role": "user", "content": "What were their major achievements?"},
    ],
    temperature=0.3,  # Lower temperature for more focused responses
    max_tokens=128,  # Reasonable length for a concise response
    top_p=0.95,  # Slightly higher for better fluency
    presence_penalty=0.2,  # Mild penalty to avoid repetition
    frequency_penalty=0.2,  # Mild penalty for more natural language
    n=1,  # Single response is usually more stable
    seed=42,  # Keep for reproducibility
)

print_highlight(response.choices[0].message.content)

[2024-11-03 07:22:58 TP0] Prefill batch. #new-seq: 1, #new-token: 48, #cached-token: 28, cache hit rate: 21.97%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-11-03 07:22:59 TP0] Decode batch. #running-req: 1, #token: 104, token usage: 0.00, gen throughput (token/s): 84.39, #queue-req: 0


[2024-11-03 07:22:59 TP0] Decode batch. #running-req: 1, #token: 144, token usage: 0.00, gen throughput (token/s): 135.70, #queue-req: 0


[2024-11-03 07:22:59 TP0] Decode batch. #running-req: 1, #token: 184, token usage: 0.00, gen throughput (token/s): 135.28, #queue-req: 0
[2024-11-03 07:22:59] INFO:     127.0.0.1:42640 - "POST /v1/chat/completions HTTP/1.1" 200 OK


Streaming mode is also supported.

In [4]:
stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

[2024-11-03 07:22:59] INFO:     127.0.0.1:42640 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2024-11-03 07:22:59 TP0] Prefill batch. #new-seq: 1, #new-token: 15, #cached-token: 25, cache hit rate: 31.40%, token usage: 0.00, #running-req: 0, #queue-req: 0
This is only a test

## Completions

### Usage

Completions API is similar to Chat Completions API, but without the `messages` parameter or chat templates.

In [5]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="List 3 countries and their capitals.",
    temperature=0,
    max_tokens=64,
    n=1,
    stop=None,
)

print_highlight(f"Response: {response}")

[2024-11-03 07:22:59 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 1, cache hit rate: 30.39%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-03 07:23:00 TP0] Decode batch. #running-req: 1, #token: 25, token usage: 0.00, gen throughput (token/s): 120.95, #queue-req: 0


[2024-11-03 07:23:00 TP0] Decode batch. #running-req: 1, #token: 65, token usage: 0.00, gen throughput (token/s): 145.87, #queue-req: 0
[2024-11-03 07:23:00] INFO:     127.0.0.1:42640 - "POST /v1/completions HTTP/1.1" 200 OK


### Parameters

The completions API accepts OpenAI Completions API's parameters.  Refer to [OpenAI Completions API](https://platform.openai.com/docs/api-reference/completions/create) for more details.

Here is an example of a detailed completions request:

In [6]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="Write a short story about a space explorer.",
    temperature=0.7,  # Moderate temperature for creative writing
    max_tokens=150,  # Longer response for a story
    top_p=0.9,  # Balanced diversity in word choice
    stop=["\n\n", "THE END"],  # Multiple stop sequences
    presence_penalty=0.3,  # Encourage novel elements
    frequency_penalty=0.3,  # Reduce repetitive phrases
    n=1,  # Generate one completion
    seed=123,  # For reproducible results
)

print_highlight(f"Response: {response}")

[2024-11-03 07:23:00 TP0] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 1, cache hit rate: 29.32%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-11-03 07:23:00 TP0] Decode batch. #running-req: 1, #token: 43, token usage: 0.00, gen throughput (token/s): 128.32, #queue-req: 0


[2024-11-03 07:23:00 TP0] Decode batch. #running-req: 1, #token: 83, token usage: 0.00, gen throughput (token/s): 137.41, #queue-req: 0


[2024-11-03 07:23:01 TP0] Decode batch. #running-req: 1, #token: 123, token usage: 0.00, gen throughput (token/s): 135.90, #queue-req: 0


[2024-11-03 07:23:01] INFO:     127.0.0.1:42640 - "POST /v1/completions HTTP/1.1" 200 OK


## Batches

Batches API for chat completions and completions are also supported. You can upload your requests in `jsonl` files, create a batch job, and retrieve the results when the batch job is completed (which takes longer but costs less).

The batches APIs are:

- `batches`
- `batches/{batch_id}/cancel`
- `batches/{batch_id}`

Here is an example of a batch job for chat completions, completions are similar.


In [7]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")

requests = [
    {
        "custom_id": "request-1",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [
                {"role": "user", "content": "Tell me a joke about programming"}
            ],
            "max_tokens": 50,
        },
    },
    {
        "custom_id": "request-2",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [{"role": "user", "content": "What is Python?"}],
            "max_tokens": 50,
        },
    },
]

input_file_path = "batch_requests.jsonl"

with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    file_response = client.files.create(file=f, purpose="batch")

batch_response = client.batches.create(
    input_file_id=file_response.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Batch job created with ID: {batch_response.id}")

[2024-11-03 07:23:01] INFO:     127.0.0.1:33666 - "POST /v1/files HTTP/1.1" 200 OK
[2024-11-03 07:23:01] INFO:     127.0.0.1:33666 - "POST /v1/batches HTTP/1.1" 200 OK
[2024-11-03 07:23:01 TP0] Prefill batch. #new-seq: 2, #new-token: 20, #cached-token: 60, cache hit rate: 42.80%, token usage: 0.00, #running-req: 0, #queue-req: 0


In [8]:
while batch_response.status not in ["completed", "failed", "cancelled"]:
    time.sleep(3)
    print(f"Batch job status: {batch_response.status}...trying again in 3 seconds...")
    batch_response = client.batches.retrieve(batch_response.id)

if batch_response.status == "completed":
    print("Batch job completed successfully!")
    print(f"Request counts: {batch_response.request_counts}")

    result_file_id = batch_response.output_file_id
    file_response = client.files.content(result_file_id)
    result_content = file_response.read().decode("utf-8")

    results = [
        json.loads(line) for line in result_content.split("\n") if line.strip() != ""
    ]

    for result in results:
        print_highlight(f"Request {result['custom_id']}:")
        print_highlight(f"Response: {result['response']}")

    print_highlight("Cleaning up files...")
    # Only delete the result file ID since file_response is just content
    client.files.delete(result_file_id)
else:
    print_highlight(f"Batch job failed with status: {batch_response.status}")
    if hasattr(batch_response, "errors"):
        print_highlight(f"Errors: {batch_response.errors}")

[2024-11-03 07:23:01 TP0] Decode batch. #running-req: 2, #token: 58, token usage: 0.00, gen throughput (token/s): 110.83, #queue-req: 0


[2024-11-03 07:23:01 TP0] Decode batch. #running-req: 1, #token: 83, token usage: 0.00, gen throughput (token/s): 171.56, #queue-req: 0


Batch job status: validating...trying again in 3 seconds...
[2024-11-03 07:23:04] INFO:     127.0.0.1:33666 - "GET /v1/batches/batch_45023f39-9421-4b18-a44f-540a9054cb13 HTTP/1.1" 200 OK
Batch job completed successfully!
Request counts: BatchRequestCounts(completed=2, failed=0, total=2)
[2024-11-03 07:23:04] INFO:     127.0.0.1:33666 - "GET /v1/files/backend_result_file-f58ad32f-d8ff-44d8-b5d6-29734fe8b98a/content HTTP/1.1" 200 OK


[2024-11-03 07:23:04] INFO:     127.0.0.1:33666 - "DELETE /v1/files/backend_result_file-f58ad32f-d8ff-44d8-b5d6-29734fe8b98a HTTP/1.1" 200 OK


It takes a while to complete the batch job. You can use these two APIs to retrieve the batch job status or cancel the batch job.

1. `batches/{batch_id}`: Retrieve the batch job status.
2. `batches/{batch_id}/cancel`: Cancel the batch job.

Here is an example to check the batch job status.

In [9]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")

requests = []
for i in range(100):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 500,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

max_checks = 5
for i in range(max_checks):
    batch_details = client.batches.retrieve(batch_id=batch_job.id)

    print_highlight(
        f"Batch job details (check {i+1} / {max_checks}) // ID: {batch_details.id} // Status: {batch_details.status} // Created at: {batch_details.created_at} // Input file ID: {batch_details.input_file_id} // Output file ID: {batch_details.output_file_id}"
    )
    print_highlight(
        f"<strong>Request counts: Total: {batch_details.request_counts.total} // Completed: {batch_details.request_counts.completed} // Failed: {batch_details.request_counts.failed}</strong>"
    )

    time.sleep(3)

[2024-11-03 07:23:04] INFO:     127.0.0.1:33682 - "POST /v1/files HTTP/1.1" 200 OK
[2024-11-03 07:23:04] INFO:     127.0.0.1:33682 - "POST /v1/batches HTTP/1.1" 200 OK


[2024-11-03 07:23:04 TP0] Prefill batch. #new-seq: 16, #new-token: 480, #cached-token: 400, cache hit rate: 44.83%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-03 07:23:04 TP0] Prefill batch. #new-seq: 84, #new-token: 2520, #cached-token: 2100, cache hit rate: 45.33%, token usage: 0.00, #running-req: 16, #queue-req: 0


[2024-11-03 07:23:05 TP0] Decode batch. #running-req: 100, #token: 6525, token usage: 0.01, gen throughput (token/s): 1083.35, #queue-req: 0


[2024-11-03 07:23:05 TP0] Decode batch. #running-req: 100, #token: 10525, token usage: 0.02, gen throughput (token/s): 11017.79, #queue-req: 0


[2024-11-03 07:23:05 TP0] Decode batch. #running-req: 100, #token: 14525, token usage: 0.03, gen throughput (token/s): 10797.59, #queue-req: 0


...

[2024-11-03 07:23:14] INFO:     127.0.0.1:36394 - "GET /v1/batches/batch_489df203-df0c-4784-aa65-efc2c19af39c HTTP/1.1" 200 OK


[2024-11-03 07:23:17] INFO:     127.0.0.1:36394 - "GET /v1/batches/batch_489df203-df0c-4784-aa65-efc2c19af39c HTTP/1.1" 200 OK


[2024-11-03 07:23:20] INFO:     127.0.0.1:36394 - "GET /v1/batches/batch_489df203-df0c-4784-aa65-efc2c19af39c HTTP/1.1" 200 OK


[2024-11-03 07:23:23] INFO:     127.0.0.1:36394 - "GET /v1/batches/batch_489df203-df0c-4784-aa65-efc2c19af39c HTTP/1.1" 200 OK


[2024-11-03 07:23:26] INFO:     127.0.0.1:36394 - "GET /v1/batches/batch_489df203-df0c-4784-aa65-efc2c19af39c HTTP/1.1" 200 OK


Here is an example to cancel a batch job.

In [10]:
import json
import time
from openai import OpenAI
import os

client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")

requests = []
for i in range(500):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 500,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

try:
    cancelled_job = client.batches.cancel(batch_id=batch_job.id)
    print_highlight(f"Cancellation initiated. Status: {cancelled_job.status}")
    assert cancelled_job.status == "cancelling"

    # Monitor the cancellation process
    while cancelled_job.status not in ["failed", "cancelled"]:
        time.sleep(3)
        cancelled_job = client.batches.retrieve(batch_job.id)
        print_highlight(f"Current status: {cancelled_job.status}")

    # Verify final status
    assert cancelled_job.status == "cancelled"
    print_highlight("Batch job successfully cancelled")

except Exception as e:
    print_highlight(f"Error during cancellation: {e}")
    raise e

finally:
    try:
        del_response = client.files.delete(uploaded_file.id)
        if del_response.deleted:
            print_highlight("Successfully cleaned up input file")
        if os.path.exists(input_file_path):
            os.remove(input_file_path)
            print_highlight("Successfully deleted local batch_requests.jsonl file")
    except Exception as e:
        print_highlight(f"Error cleaning up: {e}")
        raise e

[2024-11-03 07:23:29] INFO:     127.0.0.1:50430 - "POST /v1/files HTTP/1.1" 200 OK
[2024-11-03 07:23:29] INFO:     127.0.0.1:50430 - "POST /v1/batches HTTP/1.1" 200 OK


[2024-11-03 07:23:29 TP0] Prefill batch. #new-seq: 40, #new-token: 40, #cached-token: 2160, cache hit rate: 59.92%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-03 07:23:30 TP0] Prefill batch. #new-seq: 332, #new-token: 8192, #cached-token: 10040, cache hit rate: 56.54%, token usage: 0.01, #running-req: 40, #queue-req: 128


[2024-11-03 07:23:30 TP0] Prefill batch. #new-seq: 129, #new-token: 3868, #cached-token: 3227, cache hit rate: 54.19%, token usage: 0.03, #running-req: 371, #queue-req: 1


[2024-11-03 07:23:30 TP0] Decode batch. #running-req: 500, #token: 23025, token usage: 0.05, gen throughput (token/s): 491.30, #queue-req: 0


[2024-11-03 07:23:31 TP0] Decode batch. #running-req: 500, #token: 43025, token usage: 0.10, gen throughput (token/s): 24618.55, #queue-req: 0


...

[2024-11-03 07:23:38 TP0] Decode batch. #running-req: 500, #token: 183025, token usage: 0.41, gen throughput (token/s): 18325.06, #queue-req: 0


[2024-11-03 07:23:39 TP0] Decode batch. #running-req: 500, #token: 203025, token usage: 0.46, gen throughput (token/s): 17732.70, #queue-req: 0


[2024-11-03 07:23:39] INFO:     127.0.0.1:48598 - "POST /v1/batches/batch_60bb0890-b797-4dbd-b078-4f9ce585d597/cancel HTTP/1.1" 200 OK


[2024-11-03 07:23:42] INFO:     127.0.0.1:48598 - "GET /v1/batches/batch_60bb0890-b797-4dbd-b078-4f9ce585d597 HTTP/1.1" 200 OK


[2024-11-03 07:23:42] INFO:     127.0.0.1:48598 - "DELETE /v1/files/backend_input_file-0700bb33-a19c-4eb8-be4b-b9fbb21853ce HTTP/1.1" 200 OK


In [11]:
terminate_process(server_process)